# HyperStyle Inference Notebook

## Prepare Environment and Download HyperStyle Code

In [ ]:
#@title Clone HyperStyle Repo and Install Ninja { display-mode: "form" } 
import os
os.chdir('/content')
CODE_DIR = 'hyperstyle'

## clone repo
!git clone https://github.com/yuval-alaluf/hyperstyle.git $CODE_DIR

## install ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

os.chdir(f'./{CODE_DIR}')

In [ ]:
#@title Import Packages { display-mode: "form" } 
import time
import sys
import pprint
from tqdm import tqdm
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

import imageio
from IPython.display import HTML
from base64 import b64encode

sys.path.append(".")
sys.path.append("..")

from notebooks.notebook_utils import Downloader, HYPERSTYLE_PATHS, W_ENCODERS_PATHS, run_alignment
from utils.common import tensor2im
from utils.inference_utils import run_inversion
from utils.domain_adaptation_utils import run_domain_adaptation
from utils.model_utils import load_model, load_generator

%load_ext autoreload
%autoreload 2

## Define Download Configuration
Select below whether you wish to download all models using `pydrive`. Note that if you do not use `pydrive`, you may encounter a "quota exceeded" error from Google Drive.

In [ ]:
#@title { display-mode: "form" } 
download_with_pydrive = True #@param {type:"boolean"} 
downloader = Downloader(code_dir=CODE_DIR, use_pydrive=download_with_pydrive)

## Select Domain for Inference

In [ ]:
#@title Select which domain you wish to perform inference on: { display-mode: "form" } 
experiment_type = 'faces' #@param ['faces', 'cars', 'afhq_wild']

## Define Inference Parameters

Below we have a dictionary defining parameters such as the path to the pretrained model to use and the path to the image to perform inference on. While we provide default values to run this script, feel free to change as needed.

In [ ]:
EXPERIMENT_DATA_ARGS = {
    "faces": {
        "model_path": "./pretrained_models/hyperstyle_ffhq.pt",
        "w_encoder_path": "./pretrained_models/faces_w_encoder.pt",
        "image_path": "./notebooks/images/face_image.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
    "cars": {
        "model_path": "./pretrained_models/hyperstyle_cars.pt",
        "w_encoder_path": "./pretrained_models/cars_w_encoder.pt",
        "image_path": "./notebooks/images/car_image.jpg",
        "transform": transforms.Compose([
            transforms.Resize((192, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
    "afhq_wild": {
        "model_path": "./pretrained_models/hyperstyle_afhq_wild.pt",
        "w_encoder_path": "./pretrained_models/afhq_wild_w_encoder.pt",
        "image_path": "./notebooks/images/afhq_wild_image.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    }
}

EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS[experiment_type]

## Download Models
To reduce the number of requests to fetch the model, we'll check if the model was previously downloaded and saved before downloading the model. We'll download the model for the selected experiment and save it to the folder `hyperstyle/pretrained_models`.

We also need to verify that the model was downloaded correctly. All of our models should weigh approximately 1.3GB. Note that if the file weighs several KBs, you most likely encounter a "quota exceeded" error from Google Drive.

In [ ]:
#@title Download HyperStyle Model { display-mode: "form" } 
if not os.path.exists(EXPERIMENT_ARGS['model_path']) or os.path.getsize(EXPERIMENT_ARGS['model_path']) < 1000000:
    print(f'Downloading HyperStyle model for {experiment_type}...')
    downloader.download_file(file_id=HYPERSTYLE_PATHS[experiment_type]['id'], file_name=HYPERSTYLE_PATHS[experiment_type]['name'])
    # if google drive receives too many requests, we'll reach the quota limit and be unable to download the model
    if os.path.getsize(EXPERIMENT_ARGS['model_path']) < 1000000:
        raise ValueError("Pretrained model was unable to be downloaded correctly!")
    else:
        print('Done.')
else:
    print(f'HyperStyle model for {experiment_type} already exists!')

In [ ]:
#@title Download WEncoder Model { display-mode: "form" } 
if not os.path.exists(EXPERIMENT_ARGS['w_encoder_path']) or os.path.getsize(EXPERIMENT_ARGS['w_encoder_path']) < 1000000:
    print(f'Downloading the WEncoder model for {experiment_type}...')
    downloader.download_file(file_id=W_ENCODERS_PATHS[experiment_type]['id'], file_name=W_ENCODERS_PATHS[experiment_type]['name'])
    # if google drive receives too many requests, we'll reach the quota limit and be unable to download the model
    if os.path.getsize(EXPERIMENT_ARGS['w_encoder_path']) < 1000000:
        raise ValueError("Pretrained model was unable to be downloaded correctly!")
    else:
        print('Done.')
else:
    print(f'WEncoder model for {experiment_type} already exists!')

## Load Pretrained Model
We assume that you have downloaded all relevant models and placed them in the directory defined by the `EXPERIMENT_DATA_ARGS` dictionary.

In [ ]:
#@title Load HyperStyle Model { display-mode: "form" } 
model_path = EXPERIMENT_ARGS['model_path']
net, opts = load_model(model_path, update_opts={"w_encoder_checkpoint_path": EXPERIMENT_ARGS['w_encoder_path']})
print('Model successfully loaded!')
pprint.pprint(vars(opts))

## Define Utility Functions

In [ ]:
def generate_mp4(out_name, images, kwargs):
    writer = imageio.get_writer(out_name + '.mp4', **kwargs)
    for image in images:
        writer.append_data(image)
    writer.close()


def get_latent_and_weight_deltas(inputs, net, opts):
    opts.resize_outputs = False
    opts.n_iters_per_batch = 5
    with torch.no_grad():
        _, latent, weights_deltas, _ = run_inversion(inputs.to("cuda").float(), net, opts)
    weights_deltas = [w[0] if w is not None else None for w in weights_deltas]
    return latent, weights_deltas
    

def get_result_from_vecs(vectors_a, vectors_b, weights_deltas_a, weights_deltas_b, alpha):
    results = []
    for i in range(len(vectors_a)):
        with torch.no_grad():
            cur_vec = vectors_b[i] * alpha + vectors_a[i] * (1 - alpha)
            cur_weight_deltas = interpolate_weight_deltas(weights_deltas_a, weights_deltas_b, alpha)
            res = net.decoder([cur_vec],
                              weights_deltas=cur_weight_deltas,
                              randomize_noise=False,
                              input_is_latent=True)[0]
            results.append(res[0])
    return results

def interpolate_weight_deltas(weights_deltas_a, weights_deltas_b, alpha):
    cur_weight_deltas = []
    for weight_idx, w in enumerate(weights_deltas_a):
        if w is not None:
            delta = weights_deltas_b[weight_idx] * alpha + weights_deltas_a[weight_idx] * (1 - alpha)
        else:
            delta = None
        cur_weight_deltas.append(delta)
    return cur_weight_deltas
    
def show_mp4(filename, width):
    mp4 = open(filename + '.mp4', 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video width="%d" controls autoplay loop>
        <source src="%s" type="video/mp4">
    </video>
    """ % (width, data_url)))

## Define Input Images
Define which images to include in the animation.

In [ ]:
SEED = 42
np.random.seed(SEED)

img_transforms = EXPERIMENT_ARGS['transform']
root_dir = "./notebooks/images/animations"
image_names = ['zuckerberg', 'robbie', 'affleck', 'damon', 'stone', 'jackson', 'pitt', 'watson', 'bezos', 'kunis', 'driver', 'blunt', 'downey', 'johansson', 'dicaprio']
image_paths = [os.path.join(root_dir, image) + '.jpg' for image in image_names]

## Align Images
Align the images if needed. You can skip this step if working on non-face images or if your images are pre-aligned.

In [ ]:
#@title  { display-mode: "form" }
ALIGN_IMAGES = False #@param {type:"boolean"}

In [ ]:
# only align images if working on faces and if specified
if ALIGN_IMAGES and experiment_type == "faces":
    aligned_image_paths = []
    for image_name, image_path in zip(image_names, image_paths): 
        print(f'Aligning {image_name}...')
        aligned_image = run_alignment(image_path)
        aligned_path = os.path.join(root_dir, f'{image_name}_aligned.jpg')
        # save the aligned image
        aligned_image.save(aligned_path)
        aligned_image_paths.append(aligned_path)
        # use the save aligned images as our input image paths
        image_paths = aligned_image_paths

## Run Inference!
Invert all images and interpolate between the latent codes.

In [ ]:
#@title Run Inference! (This may take several minutes)
in_images = []
all_vecs = []
all_weights_deltas = []

if experiment_type == "cars":
    resize_amount = (512, 384)
else:
    resize_amount = (opts.output_size, opts.output_size)

for image_path in image_paths:
    print(f'Working on {os.path.basename(image_path)}...')
    original_image = Image.open(image_path)
    original_image = original_image.convert("RGB")
    input_image = img_transforms(original_image)
    # get the weight deltas for each image
    result_vec, weights_deltas = get_latent_and_weight_deltas(input_image.unsqueeze(0), net, opts)
    all_vecs.append([result_vec])
    all_weights_deltas.append(weights_deltas)
    in_images.append(original_image.resize(resize_amount))

n_transition = 25
if experiment_type == "cars":
    SIZE = 384
else:
    SIZE = opts.output_size

images = []
image_paths.append(image_paths[0])
all_vecs.append(all_vecs[0])
all_weights_deltas.append(all_weights_deltas[0])
in_images.append(in_images[0])

for i in range(1, len(image_paths)):
    if i == 0:
        alpha_vals = [0] * 10 + np.linspace(0, 1, n_transition).tolist() + [1] * 5
    else:
        alpha_vals = [0] * 5 + np.linspace(0, 1, n_transition).tolist() + [1] * 5

    for alpha in tqdm(alpha_vals):
        image_a = np.array(in_images[i - 1])
        image_b = np.array(in_images[i])
        image_joint = np.zeros_like(image_a)
        up_to_row = int((SIZE - 1) * alpha)
        if up_to_row > 0:
            image_joint[:(up_to_row + 1), :, :] = image_b[((SIZE - 1) - up_to_row):, :, :]
        if up_to_row < (SIZE - 1):
            image_joint[up_to_row:, :, :] = image_a[:(SIZE - up_to_row), :, :]

        result_image = get_result_from_vecs(all_vecs[i - 1], all_vecs[i],
                                            all_weights_deltas[i - 1], all_weights_deltas[i],
                                            alpha)[0]
        if experiment_type == "cars":
            result_image = result_image[:, 64:448, :]

        output_im = tensor2im(result_image)
        res = np.concatenate([image_joint, np.array(output_im)], axis=1)
        images.append(res)

## Save and Display Result

In [ ]:
kwargs = {'fps': 15}
save_path = "./notebooks/animations"
os.makedirs(save_path, exist_ok=True)

gif_path = os.path.join(save_path, f"{experiment_type}_gif")
generate_mp4(gif_path, images, kwargs)
show_mp4(gif_path, width=opts.output_size)